# Centroids of current streets
This script produces a "centroid" per road within the current *strassennetz_gesamt* dataset originating from the *Straßen- und Wegenetz Hamburg* WFS (https://suche.transparenz.hamburg.de/dataset/strassen-und-wegenetz-hamburg-hh-sib9).

The aim is to produce a reference point per street or place which can be used for an approximate initial georeferencing of the historical map. The historical map's reference points are represented by the centroids of the corresponding street name's bounding boxes.

We distinguish between 3 different types of roads in order to generate the centroids:
- roads consisting of exactly 1 linestring: the point at half-length of the linestring is assumed to represent the centroid
- roads consisting of exactly 2 linestrings: the interpolated point at half-length over both lines is assumed to represent the centroid
- roads consisting of more than 2 linestrings: the centroid of the linestrings' common recangular bounding box is assumed to represent the centroid.

# load libraries

In [1]:
# Load libraries
import pandas as pd
import geopandas as gpd
import ogr
from shapely import wkt, geometry, ops

# import data
The *strassennetz_gesamt* originating from the WFS dataset was exported as shapefile in QGIS (EPSG: 25832).

In [2]:
# import data
strassennetz = gpd.read_file("data/input/strassennetz_gesamt.shp")

In [3]:
# we need only a few columns (name and geometry of streets)
strassennetz = strassennetz[['strassenna','geometry']]
# rename columns
strassennetz = strassennetz.rename(columns={"strassenna":"strassenname"})

In [4]:
# delete all roads containting "Unbenannte" (=unknown roads)
# if there is this prefix within a road name
substring = "Unbenannte"
strassennetz = strassennetz[~strassennetz.strassenname.str.contains(substring)]
#strassennetz

In [5]:
# count duplicates per street name to see if a road or place consists of 1, 2, or more than 2 linestrings
duplicateroads = strassennetz.pivot_table(index = ['strassenname'], aggfunc ='size')
duplicateroads

strassenname
1. Hafenstraße       1
2. Hafenstraße       2
ABC-Straße           2
Aalheitengraben      1
Aalkrautweg          1
                    ..
Überseeallee         3
Überseeboulevard     2
Überseeplatz         1
Überseering         10
Ünner Brandheid      7
Length: 8667, dtype: int64

In [7]:
# bring those numbers to a new DataFrame
duplicateroads_df = duplicateroads.to_frame()
duplicateroads_df.reset_index(level=0, inplace=True) # create index column
duplicateroads_df.rename(columns={0: "no_of_linestrings"},inplace=True) # rename column
duplicateroads_df

,strassenname,no_of_linestrings
0,1. Hafenstraße,1
1,2. Hafenstraße,2
2,ABC-Straße,2
3,Aalheitengraben,1
4,Aalkrautweg,1
...,...,...
8662,Überseeallee,3
8663,Überseeboulevard,2
8664,Überseeplatz,1
8665,Überseering,10


## a few tests

In [9]:
# # T E S T
# hermannsburger = strassennetz.reset_index()
# hermannsburger = strassennetz[41070:41072]
# #hermannsburger['length'] = pd.Series(dtype='double')
# hermannsburger["length"] = ""
# hermannsburger

In [10]:
# # T E S T
# hermannsburger['length'] = hermannsburger['geometry'].length
# hermannsburger

## add columns

In [11]:
# add the length [m] of each linestring to *strassennetz*
strassennetz['length'] = strassennetz['geometry'].length
strassennetz

,strassenname,geometry,length
0,Wulffsgang,"LINESTRING (566877.870 5945760.400, 566868.100...",40.214836
1,Parkplatz Veringstraße,"LINESTRING (565486.050 5930387.360, 565485.850...",28.750431
2,Wittenmoor,"LINESTRING (560878.120 5938830.450, 560868.150...",75.158709
3,Parkstieg,"LINESTRING (576179.020 5939706.930, 576203.850...",134.814845
4,Heidelerchenweg,"LINESTRING (570089.760 5946900.280, 570086.850...",196.038243
...,...,...,...
42392,Reinbeker Redder,"LINESTRING (579841.420 5929954.302, 579847.840...",43.061754
42393,Hermannsburger Weg,"LINESTRING (565679.350 5920359.340, 565689.490...",104.193422
42394,Hermannsburger Weg,"LINESTRING (565780.800 5920338.844, 565781.650...",139.258051
42395,Brockesstraße,"LINESTRING (566931.490 5934038.660, 566905.350...",129.746633


## Streets and Places containing exactly one linestring

In [ ]:
data_exactly1 = []

#strassennetz['length'] = pd.Series(dtype='double')
#strassennetz['totallength'] = pd.Series(dtype='double')

for index,row in strassennetz.iterrows():
    
    for index1,row1 in duplicateroads_df.iterrows():
        
        # if a road consists of only 1 linestring ...
        if (row1['no_of_linestrings']) == 1:
            
            if row['strassenname'] == row1['strassenname']:
                
                # ... find the centroid of the road.
                midPoint = row['geometry'].interpolate(row['geometry'].length/2)
                
                data_exactly1.append(
                    {
                    'strassenname':row['strassenname'],
                    'centroid':midPoint
                    }
                )

In [18]:
df_exactly1 = pd.DataFrame(data_exactly1)
exactly1 = gpd.GeoDataFrame(df_exactly1, geometry='centroid')
# # export to shapefile
# exactly1.to_file("exactly1.shp")

## Streets and Places containing exactly two linestrings

Dissolve *strassennetz* by street name

In [19]:
strassennetz_diss = strassennetz.dissolve(by='strassenname', aggfunc='sum')
strassennetz_diss.reset_index(level=0, inplace=True)
strassennetz_diss

,strassenname,geometry,length
0,1. Hafenstraße,"LINESTRING (564488.610 5925054.160, 564490.260...",249.025777
1,2. Hafenstraße,"MULTILINESTRING ((564066.470 5925187.100, 5640...",687.222617
2,ABC-Straße,"MULTILINESTRING ((565243.370 5934352.120, 5653...",245.659887
3,Aalheitengraben,"LINESTRING (577789.200 5945238.800, 577794.960...",139.471441
4,Aalkrautweg,"LINESTRING (572992.070 5946801.660, 572972.660...",399.924837
...,...,...,...
8662,Überseeallee,"MULTILINESTRING ((566032.280 5932974.200, 5662...",472.463209
8663,Überseeboulevard,"MULTILINESTRING ((566087.550 5933235.760, 5660...",480.143858
8664,Überseeplatz,"LINESTRING (566181.650 5932665.010, 566345.710...",178.649343
8665,Überseering,"MULTILINESTRING ((567835.450 5939703.770, 5678...",1785.739041


### a few tests

In [ ]:
# dammtorstrasse = strassennetz_diss.loc[strassennetz_diss['strassenname'] == 'Dammtorstraße']
# dammtorstrasse

In [50]:
# # T E S T for Dammtorstraße
# #strassennetz_diss['geometry'].loc[strassennetz_diss['strassenname'] == 'Dammtorstraße']
# # dammtorstrasse_wkt = [g.wkt for g in dammtorstrasse['geometry'].values]
# # dammtorstrasse_wkt

# # T E S T for ALL roads
# #strassennetz_diss_wkt = [g.wkt for g in strassennetz_diss['geometry'].values]
# #strassennetz_diss_wkt

# strassennetz_diss.to_file("strassennetz_diss.shp")

### applying the tests

In [20]:
# if a road consists of exactly 2 linestrings ...

data_exactly2 = []

for index,row in strassennetz_diss.iterrows():
    
    for index1,row1 in duplicateroads_df.iterrows():
        
        if (row1['no_of_linestrings']) == 2:
        
            if row['strassenname'] == row1['strassenname']:
                
                # ... interpolate point on line.
                interpolpoint = row['geometry'].interpolate(row['geometry'].length/2)
                #print(row['strassenname'],ip.wkt)
                
                data_exactly2.append(
                    {
                    'strassenname':row['strassenname'],
                    'centroid':interpolpoint
                    }
                )

In [21]:
df_exactly2 = pd.DataFrame(data_exactly2)
exactly2 = gpd.GeoDataFrame(df_exactly2, geometry='centroid')
# # export to shapefile
# exactly2.to_file("exactly2.shp")

## Streets and Places containing three and more linestrings

In [22]:
# if a road consists of more than 2 linestrings ...

data_morethan2 = []

for index,row in strassennetz_diss.iterrows():
    
    for index1,row1 in duplicateroads_df.iterrows():
        
        if (row1['no_of_linestrings']) > 2:
        
            if row['strassenname'] == row1['strassenname']:

                # ... extract layer extent ... 
                env = row['geometry'].envelope
                #print(row['strassenname'],env)

                
                # ... and get centroid of layer extent.
                centroids = env.centroid
                
                data_morethan2.append(
                    {
                    'strassenname':row['strassenname'],
                    'centroid':centroids
                    }
                )

In [23]:
df_morethan2 = pd.DataFrame(data_morethan2)
morethan2 = gpd.GeoDataFrame(df_morethan2, geometry='centroid')
# # export to shapefile
# morethan2.to_file("morethan2.shp")

## bring the 3 created GeoDataFrames together into a common one

In [30]:
morethan2['no_linestrings']='more than 2'
exactly2['no_linestrings']='2'
exactly1['no_linestrings']='1'

# merge all 3 geodataframes
roads_centroids = pd.concat([exactly1, exactly2, morethan2])
# print GDF
roads_centroids

,strassenname,centroid,no_linestrings
0,Lampenland,POINT (581873.660 5925881.740),1
1,Achterdwars,POINT (579629.014 5927099.873),1
2,Platz am 10. Längengrad,POINT (566281.255 5932848.595),1
3,Annaberg,POINT (573372.230 5933475.120),1
4,Hein-Möller-Weg,POINT (580044.129 5927723.987),1
...,...,...,...
4981,Övelgönner Straße,POINT (562635.355 5936114.030),more than 2
4982,Övern Barg,POINT (569846.380 5943452.715),more than 2
4983,Überseeallee,POINT (566333.445 5932931.950),more than 2
4984,Überseering,POINT (567577.120 5939913.225),more than 2


In [35]:
# export to shapefile (EPSG: 25832)
roads_centroids_utm = roads_centroids.copy()
roads_centroids_utm.crs = {'init' :'epsg:25832'}
roads_centroids_utm.to_file("data/output/roads_centroids_utm.shp")